In [3]:
import pandas as pd
from prophet import Prophet   # pip install prophet
import numpy as np

df = pd.read_csv("C:/Users/karth/OneDrive/Desktop/retail-forecast-segmentation-inventory/data/processed/superstore_clean.csv")
df['Order Date'] = pd.to_datetime(df['Order Date'])
df.head()


c:\Users\karth\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


,Order Date,Customer ID,Category,Sub-Category,Product ID,Sales,Quantity,Profit,Year,Month
0,2016-11-08,CG-12520,Furniture,Bookcases,FUR-BO-10001798,261.9600,2,41.9136,2016,2016-11
1,2016-11-08,CG-12520,Furniture,Chairs,FUR-CH-10000454,731.9400,3,219.5820,2016,2016-11
2,2016-06-12,DV-13045,Office Supplies,Labels,OFF-LA-10000240,14.6200,2,6.8714,2016,2016-06
3,2015-10-11,SO-20335,Furniture,Tables,FUR-TA-10000577,957.5775,5,-383.0310,2015,2015-10
4,2015-10-11,SO-20335,Office Supplies,Storage,OFF-ST-10000760,22.3680,2,2.5164,2015,2015-10


In [4]:
# 3. Basic cleanup
df['Order Date'] = pd.to_datetime(df['Order Date'])
df = df[df['Sales'] > 0]

# 4. Aggregate to MONTHLY sales
monthly = (
    df.groupby(pd.Grouper(key='Order Date', freq='MS'))  # MS = Month Start
      ['Sales'].sum()
      .reset_index()
)

monthly.rename(columns={'Order Date': 'ds', 'Sales': 'y'}, inplace=True)

print(monthly.head())
print(monthly.tail())


          ds          y
0 2014-01-01  14236.895
1 2014-02-01   4519.892
2 2014-03-01  55691.009
3 2014-04-01  28295.345
4 2014-05-01  23648.287
           ds            y
43 2017-08-01   63120.8880
44 2017-09-01   87866.6520
45 2017-10-01   77776.9232
46 2017-11-01  118447.8250
47 2017-12-01   83829.3188


In [5]:
# 5. Fit Prophet model
m = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)
m.fit(monthly)

# 6. Make 18-month future forecast
future = m.make_future_dataframe(periods=18, freq='MS')
forecast = m.predict(future)

# 7. Merge actuals + forecast
out = monthly.merge(
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']],
    on='ds',
    how='right'
)

# 8. Flag actual vs future
out['IsActual'] = out['y'].notna()

# 9. Rename for Power BI
out.rename(columns={
    'y': 'ActualSales',
    'yhat': 'Forecast',
    'yhat_lower': 'ForecastLower',
    'yhat_upper': 'ForecastUpper'
}, inplace=True)

print(out.tail(25))


18:56:37 - cmdstanpy - INFO - Chain [1] start processing
18:56:38 - cmdstanpy - INFO - Chain [1] done processing


           ds  ActualSales       Forecast  ForecastLower  ForecastUpper  \
41 2017-06-01   52981.7257   50972.092282   42075.856047   59719.040566   
42 2017-07-01   45264.4160   48443.199794   39807.442167   57197.637622   
43 2017-08-01   63120.8880   50114.707149   41953.072993   59232.240217   
44 2017-09-01   87866.6520   88000.205994   79798.172667   96564.193816   
45 2017-10-01   77776.9232   66019.910438   57192.768644   75022.225601   
46 2017-11-01  118447.8250  100051.690150   91698.687710  108520.013659   
47 2017-12-01   83829.3188   94971.170620   86172.682992  103292.888838   
48 2018-01-01          NaN   45772.770292   36227.913255   54207.277326   
49 2018-02-01          NaN   32975.255112   24817.588752   41707.006367   
50 2018-03-01          NaN   72181.347662   63421.892972   80616.007691   
51 2018-04-01          NaN   56487.020274   47749.701190   65002.107108   
52 2018-05-01          NaN   57101.042390   49055.946684   66122.428536   
53 2018-06-01          Na

In [8]:
# 10. Save processed forecast file
save_path = r"C:/Users/karth/OneDrive/Desktop/retail-forecast-segmentation-inventory/data/processed/sales_forecast.csv"
out.to_csv(save_path, index=False)
save_path



'C:/Users/karth/OneDrive/Desktop/retail-forecast-segmentation-inventory/data/processed/sales_forecast.csv'